# Dashboard inflow Canestrari

In [2]:
import pandas as pd
import datetime as dt
import numpy as np
import os

In [ ]:
inflow_can = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Reports\Dashboard inflow canali e prodotti\Data export\inflow_canestrari.xlsx"
### Variabili
d_parser = lambda x: dt.datetime.strptime(x, '%m.%Y')
columns = ["Anno/Mese Inseriment | Anno Inserimento Ord","Ordini AP","Ordini AC","Budget AP",
           "Budget AC","Sales Director","TIPO PTR","Forecast","Sales Director infloW","linea aggregata"]


### ETL "Inflow Roberta Canestrari"
inflow = pd.read_excel(inflow_can, sheet_name="inflow", engine="openpyxl", header=3, parse_dates=["Anno/Mese Inseriment | Anno Inserimento Ord"], date_parser=d_parser, usecols=columns)

inflow.columns = ["Anno/Mese", "Ordini AP", "Ordini AC", "Budget AP", "Budget AC", "Sales Director", "Rete", "Previsioni impegno", "Area territoriale", "Linea aggregata"]
inflow.fillna({"Ordini AP":0,"Ordini AC":0,"Budget AC":0,"Budget AC":0,"Previsioni impegno":0,"Sales Director":"Missing","Rete": "Misssing","Area territoriale": "Misssing","Linea aggregata": "Misssing"}, inplace=True)
inflow["Ordini"] = inflow.loc[:,['Ordini AP','Ordini AC']].sum(axis=1, min_count=1)
inflow["Budget"]= inflow.loc[:,['Budget AP','Budget AC']].sum(axis=1, min_count=1)
inflow = inflow.loc[:,["Anno/Mese", "Sales Director", "Rete", "Area territoriale", "Linea aggregata", "Ordini", "Budget", "Previsioni impegno"]]
inflow.replace({"Area territoriale":{"Indirect Sales":"Indirect", "North-East_Ibisoft":"North-East Ibisoft"}, "Linea aggregata":{"services":"Services", "hw":"HW"}}, inplace=True)
inflow.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Reports\Dashboard inflow canali e prodotti\Source\inflow.xlsx", index=False)
gg_lav = pd.DataFrame(data=pd.date_range(start='2021-01-01', end=(dt.date.today()-pd.Timedelta(days=1))), columns=["Giorno"])
holidays=["2021-01-01","2021-01-06","2021-04-05","2021-05-01","2021-06-02","2021-08-15","2021-11-01","2021-12-08","2021-12-25","2021-12-26"]

gg_lav['N. gg lavorativi'] = np.busday_count(gg_lav["Giorno"].apply(lambda x: (pd.to_datetime(dt.date(x.year,x.month,1))) ).values.astype('datetime64[D]'), 
                                             (gg_lav['Giorno']+pd.Timedelta(days=1)).values.astype('datetime64[D]'), 
                                             weekmask=[1,1,1,1,1,0,0], holidays=holidays)
gg_lav['N. gg lavorativi nel mese'] = gg_lav.groupby(gg_lav["Giorno"].dt.month)["N. gg lavorativi"].transform("max")

In [142]:
gg_lav.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Reports\Parametriche\gg_lav.xlsx", index=False)

---

# Dashboard inflow

## Budest

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import calendar
import zipfile
from directories import *

In [2]:
zip_file =dashboard_inflow_canali_prodotti+r"\Data export\ZQ_MC_BUDANAL_IMP_ALL.zip"
directory_to_extract_to = dashboard_inflow_canali_prodotti+r"\Data export"
try:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)
except:
    print("Invalid file")

In [3]:
bw_file = dashboard_inflow_canali_prodotti+r"\Data export\ZQ_MC_BUDANAL_IMP_ALL_00000.xls"
df = pd.DataFrame(pd.read_html(str(bw_file), encoding = 'utf-8', decimal=",", thousands='.')[3])
df2 = pd.DataFrame(data=df.loc[1:,:])
df2.columns = df.loc[0,:].to_list()
df2["Data Ins. Ordine (monitoraggio)"] = pd.to_datetime(df2["Data Ins. Ordine (monitoraggio)"], format="%d%m%Y")
df2[["Raccolto", "Budget assegnato"]] = df2.loc[:,["Raccolto", "Budget assegnato"]].astype(float)
df2.loc[(df2["Budget assegnato"].notna()) & (df2["Linea Prodotti"] == "Software"), ["Gruppo Prodotti Ricl"]] = "Software"
df2.loc[(df2["Budget assegnato"].notna()) & (df2["Linea Prodotti"] == "Service on-line"), ["Gruppo Prodotti Ricl"]] = "Software"
df2.loc[(df2["Budget assegnato"].notna()) & (df2["Linea Prodotti"] == "Services"), ["Gruppo Prodotti Ricl"]] = "Vendite Services"
df2.rename(columns={'Budget assegnato': 'Budget  assegnato'}, inplace=True)

### Elenco ordini inseriti nel giorno

In [ ]:
df2_old = pd.read_excel(dashboard_inflow_prodotti+r"\inflow_prodotti_nuove_vendite_2023.xlsx")
inflow_giorno = df2.merge(df2_old, how="outer", indicator=True)
inflow_giorno = inflow_giorno.loc[inflow_giorno["_merge"] == "left_only",:]
inflow_giorno.loc[:,['Codice PRODOTTO','Gruppo Prodotti Ricl', "Agente dell'ORDINE",'Sales Director', 'Canale di VENDITA', 'Cliente Merce','Cliente FATTURA', 'Linea Prodotti', 'Numero Ordine', 'Origine Ordine','Numero Fattura', 'Data Ins. Ordine (monitoraggio)', 'Raccolto']].to_excel(dashboard_inflow_source+r"\Ordini inseriti nel giorno.xlsx", index=False)

In [ ]:
df2.to_excel(dashboard_inflow_canali_prodotti+r"\Source\inflow_prodotti\inflow_prodotti_nuove_vendite_2023.xlsx", sheet_name='Foglio1', index=False)

### Elenco ordini SAP per confronto con BPM

In [ ]:
df2.groupby(["Numero Ordine","Data Ins. Ordine (monitoraggio)"], dropna=False).agg({"Raccolto":'sum'}).reset_index().to_excel(bpm_folder+r"\Ordini_SAP.xlsx", sheet_name="Foglio1", index=False)

### Inflow mese corrente per PO Giornaliero

In [ ]:
df2_curr_month = df2.loc[df2["Data Ins. Ordine (monitoraggio)"].dt.date >= dt.date(dt.datetime.now().year,dt.datetime.now().month,1),:]

df2_curr_month.to_excel(dashboard_inflow_source+f"/po_giornaliero/inflow_prodotti_nuove_vendite_2023_mese_corrente.xlsx", sheet_name='Foglio1', index=False)

### Calcolo dei giorni lavorativi per avanzamento

In [8]:
gg_lav = pd.DataFrame(data=pd.date_range(start='2021-01-01', end=(dt.date.today()-pd.Timedelta(days=1))), columns=["Giorno"])

holidays=["2023-01-01","2023-01-06","2023-04-10","2023-04-25","2023-05-01","2023-06-02","2023-08-15","2023-11-01","2023-12-08","2023-12-25","2023-12-26"]

gg_lav['N. gg lavorativi'] = np.busday_count(
    gg_lav["Giorno"].apply(lambda x: (pd.to_datetime(dt.date(x.year,x.month,1))) ).values.astype('datetime64[D]'),
    (gg_lav['Giorno']+pd.Timedelta(days=1)).values.astype('datetime64[D]'),
    weekmask=[1,1,1,1,1,0,0], holidays=holidays)

gg_lav['N. gg lavorativi nel mese'] = np.busday_count(
    gg_lav["Giorno"].apply(lambda x: (pd.to_datetime(dt.date(x.year,x.month,1))) ).values.astype('datetime64[D]'),
    (gg_lav['Giorno'].apply(lambda d: dt.date(d.year, d.month, calendar.monthrange(d.year, d.month)[-1]))).values.astype('datetime64[D]'),
    weekmask=[1,1,1,1,1,0,0], holidays=holidays)

#gg_lav.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche\Giorni_lavorativi.xlsx", index=False)

## Grafico avanzamento verso Forecast

In [24]:
import sys
from tkinter import messagebox

### Calcolo dell'obiettivo

In [90]:
obiettivo = gg_lav.assign(Obiettivo = round(gg_lav["N. gg lavorativi"]/gg_lav["N. gg lavorativi nel mese"],2)).loc[:,"Obiettivo"].tail(1).values[0]

inflow = pd.concat([df2, 
                    pd.read_excel(dashboard_inflow_prodotti+"\\"+dashboard_inflow_prodotti_files[0])], axis=0, ignore_index=True)
forecast = pd.read_excel(dashboard_inflow_canali_prodotti+r"\Source\forecast_sales_director\forecast_sales_director_2023.xlsx")

par = {}
for file in parametriche_files:
    file_name=file.split(".")[0]
    par[f"{file_name}"] = pd.read_excel(parametriche+"\\"+file)
sales_director = pd.DataFrame(par["Sales_director"]).loc[:,["Sales Director", "Area territoriale", "Rete"]]

li_prodotti = pd.DataFrame(par["Linea_prodotti"])
esc_fatture = pd.DataFrame(par["Esclusione_fatture"])
esc_ordini = pd.DataFrame(par["Esclusione_ordini"])

inflow.rename(columns={"Data Ins. Ordine (monitoraggio)":"Giorno"}, inplace=True)
forecast.rename(columns={"Sales director":"Sales Director"}, inplace=True)

### Creazione del Dataframe in merge con il Forecast

In [ ]:
# Vecchio
df_merged = (inflow.loc[(inflow["Giorno"].astype('datetime64[M]').dt.date == dt.date.today().replace(day=1)) &
                    (~inflow["Numero Ordine"].isin(esc_ordini["Numero Ordine"])) & 
                    (~inflow["Numero Fattura"].isin(esc_fatture["Numero Fattura"])) & 
                    (inflow["Canale di VENDITA"] != "E-Commerce"),
                    ["Giorno", "Canale di VENDITA", "Numero Fattura", "Numero Ordine", "Sales Director", "Raccolto"]]\
            .merge(sales_director, how="left", on="Sales Director"))



df_avanz = df_merged.groupby([df_merged["Giorno"].astype('datetime64[M]'), "Rete", "Area territoriale"], dropna=False).agg({"Raccolto":"sum"}).reset_index()\
        .merge((forecast.merge(sales_director, how="left", on="Sales Director"))\
        .groupby(["Giorno", "Rete", "Area territoriale"], dropna=False).agg({"Forecast":"sum"}).reset_index())
    
df_avanz = pd.concat([df_avanz, df_avanz[["Raccolto","Forecast"]].agg("sum").rename("Totale").to_frame().T], axis=0)\
        .assign(Avanz = lambda x: np.where(x["Forecast"] >0,x["Raccolto"]/x["Forecast"], 0))
    
df_avanz.loc[df_avanz.index == "Totale", ["Giorno", "Rete", "Area territoriale"]] = [pd.to_datetime(dt.date.today().replace(day=1)),"Totale",""]

In [91]:
# Converti la colonna "Giorno" in tipo datetime
inflow["Giorno"] = pd.to_datetime(inflow["Giorno"], format='%Y-%m-%d')

# Ottieni il mese corrente come un oggetto Period
current_month = pd.to_datetime(dt.date.today().replace(day=1)).to_period('M')

In [92]:
df_merged = (inflow.loc[(inflow["Giorno"].dt.to_period('M') == current_month) &
                    (~inflow["Numero Ordine"].isin(esc_ordini["Numero Ordine"])) & 
                    (~inflow["Numero Fattura"].isin(esc_fatture["Numero Fattura"])) & 
                    (inflow["Canale di VENDITA"] != "E-Commerce"),
                    ["Giorno", "Canale di VENDITA", "Numero Fattura", "Numero Ordine", "Sales Director", "Raccolto"]]\
            .merge(sales_director, how="left", on="Sales Director"))

In [93]:
try:
    df_merged["Giorno"] = pd.to_datetime(df_merged["Giorno"], format='%Y-%m-%d').dt.to_period('M')
    forecast["Giorno"] = pd.to_datetime(forecast["Giorno"], format='%Y-%m-%d').dt.to_period('M')
    
    df_avanz = df_merged.groupby(["Giorno", "Rete", "Area territoriale"], dropna=False).agg({"Raccolto":"sum"}).reset_index()\
        .merge((forecast.merge(sales_director, how="left", on="Sales Director"))\
        .groupby(["Giorno", "Rete", "Area territoriale"], dropna=False).agg({"Forecast":"sum"}).reset_index())
    
    df_avanz = pd.concat([df_avanz, df_avanz[["Raccolto","Forecast"]].agg("sum").rename("Totale").to_frame().T], axis=0)\
        .assign(Avanz = lambda x: np.where(x["Forecast"] > 0, x["Raccolto"]/x["Forecast"], 0))
    
    df_avanz.loc[df_avanz.index == "Totale", ["Giorno", "Rete", "Area territoriale"]] = [pd.to_datetime(dt.date.today().replace(day=1)).to_period('M'), "Totale", ""]
    
except KeyError:
    print("Manca il forecast")
    Error=messagebox.showinfo("Manca il forecast")
    sys.exit(1)

In [94]:
import plotly.graph_objects as go
import plotly.offline as pyo

df = df_avanz.sort_values(by=["Rete", "Area territoriale"], ascending=False).copy()


# Creazione del grafico
fig = go.Figure()


fig.add_trace(go.Bar(
    y=df.loc[df["Rete"] =="Totale","Rete"] ,
    x=df.loc[df["Rete"] =="Totale", "Avanz"],
    orientation='h',
    marker=dict(color='#003566'),
    marker_line=dict(width=3, color='#219ebc'),
    name='Avanzamento',
    texttemplate='%{x:.0%}',
    textposition='inside'
))

# Aggiunta della barra orizzontale
fig.add_trace(go.Bar(
    y=df.loc[df["Rete"] !="Totale","Rete"] + " - " + df.loc[df["Rete"] !="Totale", "Area territoriale"],
    x=df.loc[df["Rete"] !="Totale", "Avanz"],
    orientation='h',
    marker=dict(color='#003566'),
    name='Avanzamento',
    texttemplate='%{x:.0%}',
    textposition='inside'
))



# Aggiunta della linea verticale dell'obiettivo
fig.add_shape(
    type="line",
    x0=obiettivo,
    y0=-0.5,
    x1=obiettivo,
    y1=len(df)-0.5,
    line=dict(
        color="red",
        width=3,
        dash="dashdot"
    )
)

# Impostazioni layout
fig.update_layout(
    title= dict(text="Avanzamento inflow vs. forecast", font_size=24),
    #xaxis_title_text="Avanzamento",
    yaxis_title_text="Rete - Area territoriale",
    yaxis=dict(automargin=True),
    xaxis=dict(tickformat=',.0%'),
    xaxis_range=[0,1],
    #category_orders={'Rete': category_order},
    height=400,
    margin=dict(l=200, r=50, t=100, b=100),
    template='plotly_white',
    showlegend=False
)
fig.show()

In [ ]:
# Mostra il grafico
fig.write_image(dashboard_inflow_canali_prodotti+r'\Avanzamento_inflow_vs_forecast.png',width=1000, height=500 )
pyo.plot(fig, filename=dashboard_inflow_canali_prodotti+f'/Avanzamento_inflow_vs_forecast.html')

## Refresh dashboard

In [1]:
import win32com.client
from PIL import ImageGrab
import time
from directories import *
import datetime as dt

In [ ]:
File = win32com.client.Dispatch("Excel.Application")
Workbook = File.Workbooks.Open(dashboard_inflow_canali_prodotti+r"\Dashboard inflow - Raffaele.xlsb")
File.Visible = True
Workbook.RefreshAll()
File.CalculateUntilAsyncQueriesDone()
Workbook.Save()

In [ ]:
Worksheet = Workbook.Worksheets("Email")

Copyrange= Worksheet.Range('B6:k109')
Copyrange.CopyPicture(Appearance=1, Format=2)

ImageGrab.grabclipboard().save(dashboard_inflow_canali_prodotti+r"\Dashboard inflow.png")

Workbook.Close()


time.sleep(5)

In [ ]:
Workbook = File.Workbooks.Open(dashboard_inflow_canali_prodotti+r"\Condivisi\Dashboard inflow.xlsb")
File.Visible = True
Workbook.RefreshAll()
File.CalculateUntilAsyncQueriesDone()
Workbook.Save()
Workbook.Close(SaveChanges=False)


time.sleep(5)

In [ ]:
workbook_path = dashboard_inflow_canali_prodotti+r"\Condivisi\Dashboard inflow - Confronto SFDC vs. SAP.xlsb"


Workbook = File.Workbooks.Open(workbook_path)
File.Visible = True
Workbook.RefreshAll()
File.CalculateUntilAsyncQueriesDone()
Workbook.Save()

In [ ]:
#Email SF vs. SAP
Sheet = Workbook.Sheets.Item("Sintesi")
Copyrange = Sheet.Range('L7:T14')
Copyrange.CopyPicture(Appearance=1, Format=2)

image_path = ImageGrab.grabclipboard().save(dashboard_inflow_canali_prodotti+r"\Condivisi\Dashboard inflow - Confronto SFDC vs. SAP.png")

Workbook.Close(SaveChanges=False)

File.Quit()

In [71]:
image_path = dashboard_inflow_canali_prodotti+r"\Condivisi\Dashboard inflow - Confronto SFDC vs. SAP.png"


html_body = """
    <div>
          <p>Buongiorno Marco,<br><br>
            qui il confronto SF vs. SAP:<br><br>
            <div>
                <img src={}></img>
            </div><br><br>
            Un saluto,<br>Raffaele<br><br></p>
    </div>
"""

outlook = win32com.client.Dispatch('Outlook.Application')
message = outlook.CreateItem(0)
message.To = "marco.bitossi@wolterskluwer.com"
message.CC = "loredana.montagna@wolterskluwer.com"
message.Subject = 'Salesforce vs SAP'
message.HTMLBody = html_body.format(image_path)

message.Display()

In [14]:
html_body = """

    <div>
        <p>
        Buongiorno,<br><br>
        in allegato la dashboard aggiornata.<br><br>
        Raffaele
        </p>


        <div><hr size="1" width="100%" align="center"></div>
        <div><hr size="1" width="100%" align="center"></div>
        <div><img width="280" height="45" src=https://cdn.wolterskluwer.io/wk/fundamentals/1.15.2/logo/assets/medium.svg></img></div>
        
        <br>
        <div><p>Aggiornamento al {}</p></div>

        <div>
        <span lang="IT">
            <a href="https://nam04.safelinks.protection.outlook.com/ap/x-59584e83/?url=https%3A%2F%2Fwolterskluwer-my.sharepoint.com%2F%3Ax%3A%2Fp%2Fraffaele_sportiello%2FEaIMGC62qmZFruYFdZFtpqUBF4RZNhrybnsO_-vTklLzTA%3Fe%3DiPyaXe&amp;data=05%7C01%7CRaffaele.Sportiello%40wolterskluwer.com%7C7656ec3191b7425e5ddf08db1e13f477%7C8ac76c91e7f141ffa89c3553b2da2c17%7C0%7C0%7C638136843317728776%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C3000%7C%7C%7C&amp;sdata=HzXyl5BWjxnXomNvjcL5qEAszwqkK%2Bo1Wp%2BHnJ3zTQY%3D&amp;reserved=0" originalsrc="https://wolterskluwer-my.sharepoint.com/:x:/p/raffaele_sportiello/EaIMGC62qmZFruYFdZFtpqUBF4RZNhrybnsO_-vTklLzTA?e=iPyaXe" shash="zaNwpHtkDDyF+cayCzaAtsK8lAqNhyt2033mQ5Y9QuS6seLu4pYbnL4SJk2JdGJ8amY+ftr1dBEzuE0R9teOdxfsbcp8U055kRtDzMbfFgQpitxl6qzXv2mHgKvRNPnKmox3RXw4xsIwrJVtvlnAlNGlZ51k4/Pzau8clAs8GFU=">
            <span lang="IT">Clicca qui per visualizzare la dashboard completa</span>
            </a>
        </span>

        <p style="  background:#007AC3;font-size:18.0pt;color:white;letter-spacing:.75pt;font-weight:bold;">
            <span lang="IT">Inflow</span>
            <span lang="IT">mese </span>
        </p>
    
        <div>
        Dashboard
        </div>
        <div>
            
        </div>

    
    </div>
"""

outlook = win32com.client.Dispatch('Outlook.Application')
message = outlook.CreateItem(0)
message.To = "raffaele.sportiello@wolterskluwer.com"
message.Subject = 'Dashboard inflow'
message.HTMLBody = html_body.format((dt.date.today()-dt.timedelta(days=1)).strftime('%d/%m/%Y'))

message.Display()

---

## PO ordini

In [16]:
import pandas as pd
import datetime as dt
import numpy as np
import zipfile
import shutil


ins_man = shutil.copy(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Source\inflow_prodotti\inflow_prodotti_ins_man.xlsx", 
            r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Source\po_giornaliero\\")

zip_file =r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Data export\ZQ_PO_TAX_GIOR.zip"
directory_to_extract_to = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Data export"
try:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)
except:
    print("Invalid file")



bw_file = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Data export\ZQ_PO_TAX_GIOR_00000.xls"
df = pd.DataFrame(pd.read_html(str(bw_file), encoding = 'utf-8', decimal=",", thousands='.')[3])
po = pd.DataFrame(data=df.loc[1:,:])
po.columns = df.loc[0,:].to_list()


po.rename(columns={"Data Ins. Ordine":"Data Ins. Ordine (monitoraggio)","Imponibile Ordine":"Raccolto"}, inplace=True)
po["Budget  assegnato"] = np.nan
po["Data Ins. Ordine (monitoraggio)"] = pd.to_datetime(po["Data Ins. Ordine (monitoraggio)"], format="%d%m%Y")
po[["Raccolto", "Budget  assegnato"]] = po.loc[:,["Raccolto", "Budget  assegnato"]].astype(float)

po.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Source\po_giornaliero\inflow_prodotti_po_giornaliero.xlsx", sheet_name='Foglio1', index=False)

---

## Classificazioni codice prodotto

In [1]:
import pandas as pd
import numpy as np
import os

In [3]:
original = pd.read_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche\Classificazione_codice_prodotto.xlsx")

In [5]:
new = pd.read_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche\Copia di modifica classificazione codici_02082022.xlsx")

In [21]:
conc = pd.concat([original,new], axis=0)
conc["Codice PRODOTTO"] = conc["Codice PRODOTTO"].astype(str)

In [28]:
conc.drop_duplicates(subset=["Codice PRODOTTO"], keep="last", inplace=True)

In [32]:
conc.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche\Classificazione_codice_prodotto - New.xlsx", index=False)

## Lista prodotti

In [60]:
import pandas as pd
import datetime as dt
import numpy as np
import zipfile
from directories import *
import os
import sys
from tkinter import messagebox

In [2]:

zip_file = dashboard_inflow+r"\Lista prodotti\ZQ_MC_BUDANAL_IMP_ALL.zip"
directory_to_extract_to = dashboard_inflow + r"\Lista prodotti"
try:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)
except:
    print("Invalid file")


In [61]:
cod_prod = pd.read_excel(parametriche+r"\Classificazione_codice_prodotto.xlsx")

In [62]:
cod_prod_inflow = pd.read_excel(dashboard_inflow_prodotti+r"\inflow_prodotti_nuove_vendite_2023.xlsx", usecols=["Codice PRODOTTO"])

In [63]:
nuovi = cod_prod_inflow.loc[~cod_prod_inflow["Codice PRODOTTO"].isin(cod_prod["Codice PRODOTTO"]),:].drop_duplicates()

In [64]:
nuovi

,Codice PRODOTTO


In [65]:
try:
    if nuovi.empty == True:
        raise RuntimeError("Il dataframe è vuoto")
#except:
 #   sys.exit(1)

AssertionError: 

: 

In [36]:
from bs4 import BeautifulSoup
import pandas as pd

with open(dashboard_inflow + r"\Lista prodotti\ZQ_MC_BUDANAL_IMP_ALL_00000.xls", 'r') as f:
    html = f.read()

soup = BeautifulSoup(html, 'html.parser')

# Recupera la tabella di interesse
table_html = str(soup.find_all('table')[8])

# Correggi il valore di colspan nella cella interessata
table_html = table_html.replace('colspan="3D2"', 'colspan="2"')

# Leggi la tabella HTML corretta con Pandas
table = pd.read_html(table_html)[0].iloc[1:,0:3]

table.columns = ["Codice PRODOTTO", "Descrizione prodotto", "Prodotto (SW)"]

In [51]:
nuovi = nuovi.merge(table, how="left", on="Codice PRODOTTO").reindex(columns=cod_prod.columns).fillna("")

In [52]:
nuovi

,Codice PRODOTTO,Descrizione prodotto,Prodotto (SW),Sub solution,Solution,Solution cluster,Business area,VSP,MDM,GTM BDG
0,BUDGETAG23F332,BUDGETAG23F332,Senza attribuz.,,,,,,,
1,BUDGETAG23G332,BUDGETAG23G332,Senza attribuz.,,,,,,,
2,BUDGETAG23Q132,BUDGETAG23Q132,Senza attribuz.,,,,,,,


In [39]:
import win32com.client

In [53]:
html_body = "<div><p>Ciao Stefania,<br><br>mi aiuti per piacere con le seguenti nuove classificazioni?:<br><br>" + nuovi.to_html(index=False) + "</div><br><br>Grazie,<br>Raffaele<br><br></p></div>"

outlook = win32com.client.Dispatch('Outlook.Application')
message = outlook.CreateItem(0)
message.To = "stefania.cremonesi@wolterskluwer.com"
message.CC = "lorella.vigato@wolterskluwer.com;martina.tisa@wolterskluwer.com;Elenamaria.Regazzoni@wolterskluwer.com;dimitri.falzone@wolterskluwer.com"
message.Subject = 'Classificazione nuovi codici prodotto'
message.HTMLBody = html_body

message.Display()